In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import DynamicEdgeConv, global_max_pool, knn_graph
import torch.optim as optim
from torch_geometric.loader import DataLoader
from tqdm import tqdm


from torch_geometric.data import Data

import os
import shutil
import numpy as np

import random


In [6]:

K = 4

In [7]:
class PointNetInstanceSeg(nn.Module):
    def __init__(self):
        super(PointNetInstanceSeg, self).__init__()
        self.edge_conv1 = DynamicEdgeConv(nn.Sequential(
            nn.Linear(14, 32),
            nn.SiLU(),
            nn.Linear(32, 64),
            nn.SiLU(),
        ), k=K)
        self.edge_conv2 = DynamicEdgeConv(nn.Sequential(
              nn.Linear(124, 64),
              nn.SiLU(),
          ), k=K)
        # self.edge_conv3 = DynamicEdgeConv(nn.Sequential(
        #       nn.Linear(256, 128),
        #       nn.SiLU(),
        #       nn.Linear(128, 64),
        #       nn.SiLU()
        #   ), k=K)
        # self.edge_conv4 = DynamicEdgeConv(nn.Sequential(
        #       nn.Linear(256, 128),
        #       nn.SiLU(),
        #       nn.Linear(128, 64),
        #       nn.SiLU()
        #   ), k=K)
        self.fc = nn.Linear(64, 2)  # Predicting instance mask for each point

    def forward(self, data):
        x, edge_index = data.pos, data.edge_index
        # print(data.edge_index)
        # return
        x = self.edge_conv1(x, edge_index)
        x = self.edge_conv2(x, edge_index)
        # x = self.edge_conv3(x, edge_index)
        # x = self.edge_conv4(x, edge_index)
        x = self.fc(x)
        return x

    # def adjust_fc(self, num_classes):
    #     self.fc = nn.Linear(128, num_classes = 2)



# Calc number of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(PointNetInstanceSeg())

50082

In [8]:

data = "/home/group10/deephalo_gnn/BinaryLabeledData/train"
files = os.listdir(data)
point_cloud_data = [(np.load(data+"/"+f)) for f in files if f.endswith(".npy")] # List of point cloud data, each element is a list of point coordinates

# Convert each point cloud data into a Data object
data_list = []
for point_cloud in point_cloud_data:
    # Create a Data object with the positions of points
    pos = torch.tensor(point_cloud[:,:-1], dtype=torch.float)
    # Recentering positions per halo
    pos[:3] = pos[:3] - pos[:3].mean(dim=1, keepdim=True)
    data = Data(
        pos=pos,
        y = torch.eye(2)[torch.tensor(point_cloud[:,-1], dtype=torch.long)],
        #edge_index=knn_graph(pos, k=K)
    )
    
    data_list.append(data)

# Now you can use DataLoader with this list of Data objects
loader = DataLoader(data_list, batch_size=1, shuffle=True)

In [9]:
weight_vec=np.array([0.000000000001,1])

In [10]:
weight_vec[0]/weight_vec[1]

1e-12

In [25]:
DEVICE = torch.device('cuda:1')

In [26]:
# Define your dataset and DataLoader
# Initialize the model
model = PointNetInstanceSeg().to(DEVICE)
weights = torch.FloatTensor(weight_vec).to(DEVICE)
# Define your loss function and optimizer
# Assuming instance masks are represented as class labels

criterion = nn.BCEWithLogitsLoss(weight=weights)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=5e-4)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for data in tqdm(loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        data.to(DEVICE)
        # print(data.y.shape)
        optimizer.zero_grad()
        outputs = model(data)
        # Assuming instance masks are represented as class labels and provided in data.y
        # target = torch.argmax(data.y, dim=1)  # Convert one-hot encoded target to class labels
        # loss = criterion(outputs, target)
        loss = criterion(outputs, data.y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * data.num_graphs
    
    epoch_loss = running_loss / len(loader.dataset)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Create the "ckpts" directory if it doesn't exist
import os, time

os.makedirs("ckpts", exist_ok=True)
curr_time = time.strftime("%Y%m%d-%H%M%S")
model_name = f"{curr_time}_pointnet_instance_seg"

# Save the model
torch.save(model.state_dict(), f"./ckpts/{model_name}_model.pth")

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


: 

In [ ]:
data_test = "/home/group10/deephalo_gnn/BinaryLabeledData/test"
files = os.listdir(data_test)
point_cloud_data = [(np.load(data_test+"/"+f)) for f in files if f.endswith(".npy")] # List of point cloud data, each element is a list of point coordinates

# Convert each point cloud data into a Data object
data_test_list = []
for point_cloud in point_cloud_data:
    # Create a Data object with the positions of points
    data_test = Data(pos=torch.tensor(point_cloud[:,:-1], dtype=torch.float), y = torch.eye(2)[torch.tensor(point_cloud[:,-1], dtype=torch.long)])
    
    data_test_list.append(data_test)


In [ ]:

test_loader = DataLoader(data_test_list, batch_size=1, shuffle=False)

# Put the model in evaluation mode

    

model.eval()

# Initialize a list to store the predictions
ground_truth_labels = []
predictions = []

# Loop over the test data
with torch.no_grad():
    for data in tqdm(test_loader, desc='Testing'):
        # Move data to the device
        data = data.to(DEVICE)
        
        # Pass the data through the model
        outputs = model(data)
        
        # Get the predicted labels
        _, predicted_labels = torch.max(outputs, 1)
        
        # Store the predictions
        ground_truth_labels.append(data.y.cpu().numpy())
        predictions.append(predicted_labels.cpu().numpy())



Testing: 100%|██████████| 30/30 [00:02<00:00, 10.04it/s]


In [ ]:
from sklearn.metrics import accuracy_score, f1_score


# Calculate accuracy
accs = []
f1s = []
for idx, (gt, pred) in enumerate(zip(ground_truth_labels, predictions)):
    # Checking if the model predicts different labels for different points in the same point cloud
    # if np.unique(pred).shape[0] != 1:
    print(idx, "\t", gt.shape, "\t", np.unique(pred), "\t", np.unique(gt))
    print()
    
    # accs.append(accuracy_score(gt, pred))
    # f1s.append(f1_score(gt, pred, average='weighted'))

print(f"Mean acc: {np.mean(accs):.4f} \pm {np.std(accs):.4f}")
print(f"Mean F!: {np.mean(f1s):.4f} \pm {np.std(f1s):.4f}")

0 	 (8668, 2) 	 [0] 	 [0. 1.]

1 	 (176809, 2) 	 [0] 	 [0. 1.]

2 	 (7641, 2) 	 [0] 	 [0. 1.]

3 	 (5704, 2) 	 [0] 	 [0. 1.]

4 	 (7083, 2) 	 [0] 	 [0. 1.]

5 	 (10423, 2) 	 [0] 	 [0. 1.]

6 	 (31397, 2) 	 [0] 	 [0. 1.]

7 	 (9118, 2) 	 [0] 	 [0. 1.]

8 	 (38128, 2) 	 [0] 	 [0. 1.]

9 	 (13188, 2) 	 [0] 	 [0. 1.]

10 	 (12394, 2) 	 [0] 	 [0. 1.]

11 	 (9524, 2) 	 [0] 	 [0. 1.]

12 	 (7714, 2) 	 [0] 	 [0. 1.]

13 	 (12774, 2) 	 [0] 	 [0. 1.]

14 	 (8716, 2) 	 [0] 	 [0. 1.]

15 	 (5366, 2) 	 [0] 	 [0. 1.]

16 	 (13644, 2) 	 [0] 	 [0. 1.]

17 	 (18205, 2) 	 [0] 	 [0. 1.]

18 	 (15889, 2) 	 [0] 	 [0. 1.]

19 	 (7831, 2) 	 [0] 	 [0. 1.]

20 	 (69640, 2) 	 [0] 	 [0. 1.]

21 	 (14424, 2) 	 [0] 	 [0. 1.]

22 	 (5642, 2) 	 [0] 	 [0. 1.]

23 	 (90050, 2) 	 [0] 	 [0. 1.]

24 	 (22845, 2) 	 [0] 	 [0. 1.]

25 	 (5874, 2) 	 [0] 	 [0. 1.]

26 	 (16718, 2) 	 [0] 	 [0. 1.]

27 	 (10287, 2) 	 [0] 	 [0. 1.]

28 	 (10569, 2) 	 [0] 	 [0. 1.]

29 	 (5575, 2) 	 [0] 	 [0. 1.]

Mean acc: nan \pm nan
Mean F!: n